In [ ]:
# @title auto-lyrics
# Install necessary packages
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git

import subprocess
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import output  # Ensure this import is included

def download_and_transcribe(urls):
    urls = urls.strip().split('\n')
    for i, url in enumerate(urls, 1):
        if url.strip():
            audio_file = f"audio_{i}"
            print(f"Downloading audio from: {url}")
            # Download the audio using yt-dlp
            yt_dlp_command = f"yt-dlp -x -o '{audio_file}.%(ext)s' --force-overwrites '{url}'"
            subprocess.run(yt_dlp_command, shell=True, check=True)
            print(f"Transcribing audio: {audio_file}.opus")
            # Transcribe the audio using Whisper via subprocess
            whisper_command = f"whisper {audio_file}.opus --model large"
            result = subprocess.run(whisper_command, shell=True, capture_output=True, text=True)
            transcription = result.stdout
            # Check if transcription needs decoding
            if isinstance(transcription, bytes):
                transcription = transcription.decode('utf-8')
            # Print and optionally save the transcription
            print(f"Transcription for {url}: {transcription}\n")
            with open(f"transcription_{i}.txt", "w") as f:
                f.write(transcription)

# Create text area widget for input
clear_output(wait=True)
text_area = widgets.Textarea(
    value='',
    placeholder='Enter YouTube URLs separated by new lines',
    description='URLs:',
    layout={'width': '100%', 'height': '200px'}
)

# Create a button to trigger processing
process_button = widgets.Button(description="Process URLs")

# Set the button's on-click event to process the URLs
def on_button_clicked(b):
    with output.use_tags('output'):
        clear_output(wait=True)
        download_and_transcribe(text_area.value)

process_button.on_click(on_button_clicked)

# Display the widgets
display(text_area, process_button)
